In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Reshape, Dropout, Dense 
from tensorflow.keras.layers import Flatten, BatchNormalization
from tensorflow.keras.layers import Activation, ZeroPadding2D
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import UpSampling2D, Conv2D
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
from tqdm import tqdm
import os 
import time
import matplotlib.pyplot as plt

In [2]:
translate_list = [' ',
             '1',
             '2',
             '3',
             '4',
             '5',
             '6',
             '7',
             '8',
             '9',
             '0',
             '+',
             '-',
             '*',
             '/',
             '=',
            ]

In [3]:
CHAR_LIST_LEN = len(translate_list)
PROBLEM_LENGTH = 5
PROBLEM_DIM = PROBLEM_LENGTH * CHAR_LIST_LEN

SEED_SIZE = 1000

N_EPOCHS = 20
BATCH_SIZE = 50
BUFFER_SIZE = 60000

In [4]:
def prob_to_array(in_string):
    problem_array = []
    for char in in_string:
        value = translate_list.index(char)
        char_array = [0]*CHAR_LIST_LEN
        char_array[value] = 1
        problem_array.append(char_array)
    if len(in_string) < PROBLEM_LENGTH:
        for _ in range(PROBLEM_LENGTH-len(in_string)):
            char_array = [0]*CHAR_LIST_LEN
            char_array[0] = 1
            problem_array.append(char_array)
    return problem_array

def array_to_prob(problem_array):
    problem_string = ""
    for char_array in problem_array:
        largest = max(char_array)
        value = char_array.index(largest)
        char = translate_list[value]
        problem_string += char
    problem_string.strip()
    return problem_string

#myarr = prob_to_array('90+21')
#myprob = array_to_prob(myarr)
#print(myarr)
#print(myprob)


In [5]:
import random
def generate_input(count):
    output_list = []
    for _ in range(count):
        left = random.randint(10,99)
        right = random.randint(10,99)
        problem = str(left) + '+' + str(right)
        output_list.append(problem)
    return output_list

In [6]:
input_problems = generate_input(1000)
# print(input_problems)
training_data = []
for item in input_problems:
    temp_arr = prob_to_array(item)
    training_data.append(temp_arr)
training_data = np.asarray(training_data)
training_data = training_data.astype(np.float32)
print("Shape: " + str(training_data.shape))

Shape: (1000, 5, 16)


In [7]:
train_dataset = tf.data.Dataset.from_tensor_slices(training_data) \
    .shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [8]:
generator = Sequential([
    Dense(64,activation="relu",input_dim=SEED_SIZE),
    Dense(128, activation="selu"),
    #model.add(Dense(28*28, activation="sigmoid")),
    Dense(PROBLEM_LENGTH*CHAR_LIST_LEN, activation="sigmoid"),
    Reshape([PROBLEM_LENGTH, CHAR_LIST_LEN])
])

discriminator = Sequential([
    Flatten(input_shape=[PROBLEM_LENGTH, CHAR_LIST_LEN]),
    Dense(128, activation="selu"),
    Dense(64, activation="selu"),
    Dense(1, activation="sigmoid")
])

In [9]:
discriminator.compile(loss="binary_crossentropy", optimizer="rmsprop")
gan = Sequential([generator, discriminator])
gan.compile(loss="binary_crossentropy", optimizer="rmsprop")

In [10]:
def display_problems(fake_problems):
    numpy_array = fake_problems.numpy()
    # print(numpy_array)
    for prob_array in numpy_array:
        py_array = prob_array.tolist()
        prob_string = array_to_prob(py_array)
        print(prob_string)
    # print(fake_problems.shape)

In [11]:

"""
Generator not random enough
Sometimes there is one character that persists when it shouldnt
Maybe increase size of random input
"""
for epoch in range(N_EPOCHS):
    for real_problems in train_dataset:
        print("------------------------------")
        # print("Real Problems: " + str(real_problems.shape))
        noise = tf.random.normal(shape=[BATCH_SIZE, SEED_SIZE])
        # print("Noise: " + str(noise.shape))
        fake_problems = generator(noise)
        #print("Real: ")
        #display_problems(real_problems)
        print("Fakes: ")
        display_problems(fake_problems)
        mixed_problems = tf.concat([fake_problems, real_problems], axis=0)
        # print("Mixed problems: " + str(mixed_problems.shape))
        discriminator_labels = tf.constant([[0.]] * BATCH_SIZE + [[1.]] * BATCH_SIZE)
        # print("Discrim labels: " + str(discriminator_labels.shape))
        discriminator.trainable = True
        print("Discriminator loss: " + str(discriminator.train_on_batch(mixed_problems, discriminator_labels)))
        #discriminator.train_on_batch(mixed_problems, discriminator_labels)
        #print("Discriminator summary: " + str(discriminator.summary()))
        noise = tf.random.normal(shape=[BATCH_SIZE, SEED_SIZE])
        generator_labels = tf.constant([[1.]] * BATCH_SIZE)
        discriminator.trainable = False
        print("Generator loss: " + str(gan.train_on_batch(noise, generator_labels)))
        #gan.train_on_batch(noise, generator_labels)
        # print("GAN summary: " + str(gan.summary()))

------------------------------
Fakes: 
 4+*-
*+175
2=+/9
 3=56
  =47
+/*59
4/616
2-+3 
*261-
/=672
 =+86
+2- 9
 /769
24-76
2 =76
9 =9-
 3=16
8118/
83=7=
 120=
24=/9
+ 292
 =759
*1=+=
7 =1-
 +*06
 +259
29+46
 1+46
14289
 4*79
 /*73
21+39
27+89
8*=7/
 1+36
 +125
2 +39
/-07+
* =3=
-3/3-
*-=-6
+/=5-
  =76
  +/9
*4375
24312
*0=47
 +202
  =7/
Discriminator loss: 0.6533718705177307
Generator loss: 4.422799587249756
------------------------------
Fakes: 
9/=37
9/=47
- =18
 1630
73397
 9646
84/49
 7=4/
/ =93
27+19
2 /39
91=0=
 =357
21=87
-3/5-
1 -17
2 +56
10=66
9=707
1 -56
2/+36
+//51
41-16
+0=16
 /=52
2/286
83-36
2/202
24+19
3 /1-
9/649
2=*52
82*26
37-44
24=76
21==-
54+9-
+/=37
94-36
2 =36
9/=16
1 =8=
1 +3-
24/1 
*7352
2-+8-
9==86
2 /1-
27+33
82=3 
Discriminator loss: 0.7939853072166443
Generator loss: 2.876323938369751
------------------------------
Fakes: 
+/==-
+0+59
0400-
8*-19
5 +/6
+/-59
0/+56
91=04
21=1-
21/=-
+4+5-
23++6
01334
-/=//
00-56
 0-44
03==-
1 +16
20=16
-/35+
8 -5-
91=5-
+7+33

Discriminator loss: 0.7557043433189392
Generator loss: 0.6834869384765625
------------------------------
Fakes: 
9085-
90+5-
90+5-
90+5-
90+5-
90+5-
20+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
00+5-
90+/-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
00+5-
90+/-
90+5-
90+5-
90+5-
90+5-
90+5-
90+/-
90+5-
00+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
Discriminator loss: 0.7443015575408936
Generator loss: 0.6434906125068665
------------------------------
Fakes: 
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90/5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+/-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
9085-
90+5-
00+5-
90+5-
Discriminator loss: 0.739898681640625
Generator loss: 0.6253277659416199
------------------------------
Fakes: 
90+/-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
90+5-
20+5-
90+5-
90

Generator loss: 0.43315762281417847
------------------------------
Fakes: 
82+47
82+47
85+47
88+47
82+47
85+40
82+47
82+47
82+47
85+47
82+47
82+47
87+30
86+47
82+47
82+47
82+40
82+47
86+47
82+47
89+47
89+87
82+40
82+47
86+44
89+47
87+47
89+30
87+47
88+47
+6+40
82+47
86+47
82+47
86+47
82+47
86+40
85+47
87+40
82+47
82+47
82+40
82+47
89+47
87+47
82+47
82+47
82+47
86+47
82+77
Discriminator loss: 0.8528280854225159
Generator loss: 0.4252825081348419
------------------------------
Fakes: 
87+47
85+47
85+37
82+42
86+47
85+47
85+40
82+40
85+47
87+47
85+47
87+47
88+47
86+47
85+41
82+47
82+60
88+47
85+40
82+47
87+47
87+37
87+47
88+47
89+97
86+41
87+77
86+47
86+47
82+42
82+41
85+30
12+47
87+40
82+90
86+47
85+47
87+47
85+47
82+40
82+47
85+47
87+67
87+41
87+47
87+77
82+37
82+37
82+47
82+47
Discriminator loss: 0.8646323680877686
Generator loss: 0.4150114059448242
------------------------------
Fakes: 
86+97
82+95
82+60
88+90
85+47
85+47
82+37
87+45
85+47
85+47
85+47
85+47
82+97
87+67
82+45
82+67
85+

Generator loss: 0.33992329239845276
------------------------------
Fakes: 
53+65
53+65
53+65
53+65
53+65
53+65
53+65
73+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
43+65
77+65
53+65
53+65
53+65
53+65
53+65
53+65
58+65
53+65
53+69
53+65
53+65
53+65
53+65
73+65
53+85
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+85
53+65
53+65
Discriminator loss: 1.0167009830474854
Generator loss: 0.3419845700263977
------------------------------
Fakes: 
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
73+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
43+65
73+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
73+65
53+65
53+65
53+65
53+65
Discriminator loss: 1.019983172416687
Generator loss: 0.3404861390590668
------------------------------
Fakes: 
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
53+65
43+65
53+65
53+65
53+65
73+65
53+6

Discriminator loss: 1.0996795892715454
Generator loss: 0.3108343482017517
------------------------------
Fakes: 
53+63
53+63
53+63
53+63
53+63
53+63
53+61
53+63
23+43
53+68
53+63
53+63
53+63
53+63
53+64
53+63
53+61
53+63
53+63
53+63
53+61
53+51
53+61
53+64
53+63
53+63
53+63
53+64
53+64
53+63
53+63
53+63
53+68
53+63
53+63
23+64
53+64
53+53
23+61
53+63
53+53
53+63
53+63
53+63
53+61
53+63
53+63
53+53
53+63
53+63
Discriminator loss: 1.110317587852478
Generator loss: 0.3135480284690857
------------------------------
Fakes: 
53+54
53+63
53+64
53+64
53+63
53+61
53+63
53+44
53+53
53+64
53+53
53+64
53+53
53+64
53+63
53+63
53+63
33+64
53+53
53+64
53+63
53+64
53+63
53+63
53+63
53+64
53+63
53+63
53+53
53+64
53+64
53+63
53+53
53+64
23+63
53+63
53+63
53+64
53+53
53+64
53+63
53+63
53+63
53+43
53+63
53+63
53+61
53+63
53+63
53+53
Discriminator loss: 1.1089192628860474
Generator loss: 0.3158523440361023
------------------------------
Fakes: 
53+53
53+63
53+53
23+63
53+53
53+63
53+51
53+53
53+64
53+54
53

...

Generator loss: 0.19548554718494415
------------------------------
Fakes: 
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
96+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
28+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
96+34
26+34
26+34
Discriminator loss: 1.1942459344863892
Generator loss: 0.19306917488574982
------------------------------
Fakes: 
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
28+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
Discriminator loss: 1.2026305198669434
Generator loss: 0.20114080607891083
------------------------------
Fakes: 
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
26+34
2

Discriminator loss: 1.238650918006897
Generator loss: 0.1840985119342804
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
Discriminator loss: 1.2405157089233398
Generator loss: 0.1838056892156601
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
Discriminator loss: 1.2402061223983765
Generator loss: 0.18476133048534393
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
9

Generator loss: 0.17976367473602295
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
99+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
91+34
90+34
90+34
99+34
90+34
90+34
90+34
90+34
90+34
90+34
Discriminator loss: 1.2363662719726562
Generator loss: 0.1827974170446396
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+31
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
91+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
Discriminator loss: 1.2351906299591064
Generator loss: 0.1831987053155899
------------------------------
Fakes: 
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+34
90+

Generator loss: 0.17778104543685913
------------------------------
Fakes: 
40+14
40+64
40+64
40+64
40+14
40+64
40+64
40+14
40+64
40+14
40+64
40+14
40+14
40+15
40+64
40+64
40+64
40+64
40+14
40+14
40+14
40+64
40+14
40+64
40+64
40+14
40+64
40+14
40+64
40+64
40+64
40+64
40+34
45+14
40+64
40+14
40+64
40+14
40+14
45+64
40+14
40+64
40+64
40+14
40+14
40+64
40+14
45+64
40+64
40+14
Discriminator loss: 1.239141583442688
Generator loss: 0.17828333377838135
------------------------------
Fakes: 
40+14
40+14
40+14
40+14
45+64
45+64
40+64
45+14
45+14
45+64
40+64
45+64
40+14
40+64
40+64
40+64
40+14
40+64
40+64
40+64
40+64
40+64
45+65
40+64
40+14
40+64
45+64
40+64
40+64
40+64
45+14
45+65
40+34
40+61
40+64
40+64
40+64
40+14
40+64
40+64
40+61
40+14
40+64
45+64
40+64
45+64
45+84
45+65
40+64
40+61
Discriminator loss: 1.2379764318466187
Generator loss: 0.17847250401973724
------------------------------
Fakes: 
40+14
40+14
40+11
45+14
40+65
40+65
45+64
40+64
45+14
40+14
40+65
45+15
40+64
40+61
40+15
40+15
40

------------------------------
Fakes: 
57+25
57+55
57+55
57+25
57+55
57+55
57+65
57+55
57+55
57+55
57+25
57+25
57+55
57+55
57+25
57+55
57+55
57+65
57+55
57+25
57+55
57+55
57+55
57+55
57+25
57+55
57+25
57+55
57+55
57+25
57+25
57+25
57+25
57+55
57+55
57+25
57+25
57+25
57+65
57+25
57+25
57+25
57+25
57+55
57+55
17+25
57+25
57+55
57+65
57+25
Discriminator loss: 1.253576636314392
Generator loss: 0.16737203299999237
------------------------------
Fakes: 
57+55
57+55
57+25
57+55
57+55
57+65
57+65
57+55
57+45
57+55
57+55
57+55
57+25
57+55
57+55
57+55
57+25
57+55
57+55
57+45
57+55
57+25
57+25
57+65
57+55
57+55
57+55
57+25
57+55
57+65
57+55
57+65
57+25
57+25
57+55
57+25
57+55
57+55
57+25
57+55
57+65
57+55
57+25
57+25
57+25
57+65
57+25
57+55
57+55
57+55
Discriminator loss: 1.2551401853561401
Generator loss: 0.16832377016544342
------------------------------
Fakes: 
57+55
57+55
57+55
57+55
57+25
57+55
57+25
57+25
57+55
57+55
57+25
57+25
57+25
57+55
57+25
57+55
57+65
57+25
57+25
57+25
57+55
57+55
57